# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse         Shipments  \
0  Oct 11 2022  8:33AM  248322        19  TR-RCVC-10102022   
1  Oct 11 2022  8:32AM  248321        10        0086068192   
2  Oct 11 2022  8:32AM  248321        10        0086068218   
3  Oct 11 2022  8:32AM  248321        10        0086068219   
4  Oct 11 2022  8:32AM  248321        10        0086068220   
5  Oct 11 2022  8:32AM  248321        10        0086068313   
6  Oct 11 2022  8:32AM  248321        10        0086068316   
7  Oct 11 2022  8:32AM  248321        10        0086068317   
8  Oct 11 2022  8:32AM  248321        10        0086068320   
9  Oct 11 2022  8:32AM  248321        10        0086068354   

                 Customer ShipmentStatus  
0  GCH Granules USA, Inc.       Released  
1       ISDIN Corporation       Released  
2       ISDIN Corporation       Released  
3       ISDIN Corporation       Released  
4       ISDIN Corporation       Released  
5       ISDIN Corporation       Released  
6       ISDIN Corporation       Released  
7       ISDIN Corporation       Released  
8       ISDIN Corporation       Released  
9       ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
22  248317       Released          1
23  248318       Released          5
24  248319       Released          7
25  248321       Released         12
26  248322       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
248317                NaN        NaN       1.0
248318                NaN        NaN       5.0
248319                NaN        NaN       7.0
248321                NaN        NaN      12.0
248322                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
248034                0.0        0.0       1.0
248198                0.0        1.0       0.0
248200                0.0        1.0       0.0
248216                0.0        0.0       1.0
248244                0.0        0.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
248034                  0          0         1
248198                  0          1         0
248200                  0          1         0
248216                  0          0         1
248244                  0          0         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               248034          0          0         1
1               248198          0          1         0
2               248200          0          1         0
3               248216          0          0         1
4               248244          0          0         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               248034                            1
1               248198                   1         
2               248200                   1         
3               248216                            1
4               248244                            3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                      Customer
0   Oct 11 2022  8:33AM  248322        19        GCH Granules USA, Inc.
1   Oct 11 2022  8:32AM  248321        10             ISDIN Corporation
13  Oct 11 2022  8:30AM  248309        10          Lupin Research, Inc.
14  Oct 11 2022  8:30AM  248311        10                CLINUVEL, Inc.
15  Oct 11 2022  8:27AM  248319        10  Snap Medical Industries, LLC
22  Oct 11 2022  8:22AM  248317        18          Emersa Waterbox, LLC
23  Oct 11 2022  8:22AM  248316        16          Emersa Waterbox, LLC
24  Oct 11 2022  8:18AM  248318        10                   Emerginnova
29  Oct 11 2022  8:11AM  248315        20                Exact-Rx, Inc.
41  Oct 11 2022  7:46AM  248314        19           AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                      Customer  \
0  Oct 11 2022  8:33AM  248322        19        GCH Granules USA, Inc.   
1  Oct 11 2022  8:32AM  248321        10             ISDIN Corporation   
2  Oct 11 2022  8:30AM  248309        10          Lupin Research, Inc.   
3  Oct 11 2022  8:30AM  248311        10                CLINUVEL, Inc.   
4  Oct 11 2022  8:27AM  248319        10  Snap Medical Industries, LLC   
5  Oct 11 2022  8:22AM  248317        18          Emersa Waterbox, LLC   
6  Oct 11 2022  8:22AM  248316        16          Emersa Waterbox, LLC   
7  Oct 11 2022  8:18AM  248318        10                   Emerginnova   
8  Oct 11 2022  8:11AM  248315        20                Exact-Rx, Inc.   
9  Oct 11 2022  7:46AM  248314        19           AdvaGen Pharma, Ltd   

  Completed Executing Released  
0                            1  
1                           12  
2                            1  
3                            1  
4                            7  
5                            1  
6                            1  
7                            5  
8                   6        6  
9                            3

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                      Customer  \
0  Oct 11 2022  8:33AM  248322        19        GCH Granules USA, Inc.   
1  Oct 11 2022  8:32AM  248321        10             ISDIN Corporation   
2  Oct 11 2022  8:30AM  248309        10          Lupin Research, Inc.   
3  Oct 11 2022  8:30AM  248311        10                CLINUVEL, Inc.   
4  Oct 11 2022  8:27AM  248319        10  Snap Medical Industries, LLC   
5  Oct 11 2022  8:22AM  248317        18          Emersa Waterbox, LLC   
6  Oct 11 2022  8:22AM  248316        16          Emersa Waterbox, LLC   
7  Oct 11 2022  8:18AM  248318        10                   Emerginnova   
8  Oct 11 2022  8:11AM  248315        20                Exact-Rx, Inc.   
9  Oct 11 2022  7:46AM  248314        19           AdvaGen Pharma, Ltd   

  Released Executing Completed  
0        1                      
1       12                      
2        1                      
3        1                      
4        7                      
5        1                      
6        1                      
7        5                      
8        6         6            
9        3

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Oct 11 2022  8:33AM  248322        19        GCH Granules USA, Inc.   
1  Oct 11 2022  8:32AM  248321        10             ISDIN Corporation   
2  Oct 11 2022  8:30AM  248309        10          Lupin Research, Inc.   
3  Oct 11 2022  8:30AM  248311        10                CLINUVEL, Inc.   
4  Oct 11 2022  8:27AM  248319        10  Snap Medical Industries, LLC   

  Released Executing Completed  
0        1                      
1       12                      
2        1                      
3        1                      
4        7

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Oct 11 2022  8:33AM  248322        19        GCH Granules USA, Inc.   
1  Oct 11 2022  8:32AM  248321        10             ISDIN Corporation   
2  Oct 11 2022  8:30AM  248309        10          Lupin Research, Inc.   
3  Oct 11 2022  8:30AM  248311        10                CLINUVEL, Inc.   
4  Oct 11 2022  8:27AM  248319        10  Snap Medical Industries, LLC   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1      12.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4       7.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Oct 11 2022  8:33AM  248322        19        GCH Granules USA, Inc.   
1  Oct 11 2022  8:32AM  248321        10             ISDIN Corporation   
2  Oct 11 2022  8:30AM  248309        10          Lupin Research, Inc.   
3  Oct 11 2022  8:30AM  248311        10                CLINUVEL, Inc.   
4  Oct 11 2022  8:27AM  248319        10  Snap Medical Industries, LLC   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1      12.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4       7.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1738119      30.0        0.0        0.0
12          248312       1.0        0.0        0.0
15          496522       2.0        1.0       44.0
16          248316       1.0        0.0        0.0
18          248317       1.0        0.0        0.0
19          993253       5.0        1.0        0.0
20          496611       7.0        6.0        0.0
21         1240919       3.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1738119      30.0        0.0        0.0
1        12   248312       1.0        0.0        0.0
2        15   496522       2.0        1.0       44.0
3        16   248316       1.0        0.0        0.0
4        18   248317       1.0        0.0        0.0
5        19   993253       5.0        1.0        0.0
6        20   496611       7.0        6.0        0.0
7        21  1240919       3.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      30.0        0.0        0.0
1        12       1.0        0.0        0.0
2        15       2.0        1.0       44.0
3        16       1.0        0.0        0.0
4        18       1.0        0.0        0.0
5        19       5.0        1.0        0.0
6        20       7.0        6.0        0.0
7        21       3.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   30.0
1        12  Released    1.0
2        15  Released    2.0
3        16  Released    1.0
4        18  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16   18   19   20   21
Status                                             
Completed   0.0  0.0  44.0  0.0  0.0  0.0  0.0  0.0
Executing   0.0  0.0   1.0  0.0  0.0  1.0  6.0  2.0
Released   30.0  1.0   2.0  1.0  1.0  5.0  7.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16   18   19   20   21
0          Completed   0.0  0.0  44.0  0.0  0.0  0.0  0.0  0.0
1          Executing   0.0  0.0   1.0  0.0  0.0  1.0  6.0  2.0
2           Released  30.0  1.0   2.0  1.0  1.0  5.0  7.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16   18   19   20   21
0  Completed   0.0  0.0  44.0  0.0  0.0  0.0  0.0  0.0
1  Executing   0.0  0.0   1.0  0.0  0.0  1.0  6.0  2.0
2   Released  30.0  1.0   2.0  1.0  1.0  5.0  7.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()